## RDD Transformation

1. **map(func)**
1. **filter(func)**
1. **flatMap(func)**
1. **mapPartitions(func)**
1. **mapPartitionsWithIndex(func)**
1. **union(otherDataset)**
1. **intersection(otherDataset)**
1. **distinct([numPartitions])**
1. **groupByKey([numPartitions])**
1. **reduceByKey(func, [numPartitions])**
1. **sortByKey([ascending], [numPartitions])**
1. **join(otherDataset, [numPartitions])**
1. **coalesce(numPartitions)**
1. **repartition(numPartitions)**

In [0]:
sc

<SparkContext master=local[*, 4] appName=Databricks Shell>

### Map
Return a new distributed dataset formed by passing each element of the source through a function func.

In [0]:
source_data = '/FileStore/rdd/trans/'

In [0]:
sales = sc.textFile(source_data + '2019.csv')

In [0]:
sales.collect()

[['SalesOrderNumber',
  'SalesOrderLineNumber',
  'OrderDate',
  'CustomerName',
  'EmailAddress',
  'Item',
  'Quantity',
  'UnitPrice',
  'TaxAmount'],
 ['SO43701',
  '1',
  '2019-07-01',
  'Christy Zhu',
  'christy12@adventure-works.com',
  '"Mountain-100 Silver',
  ' 44"',
  '1',
  '3399.99',
  '271.9992'],
 ['SO43704',
  '1',
  '2019-07-01',
  'Julio Ruiz',
  'julio1@adventure-works.com',
  '"Mountain-100 Black',
  ' 48"',
  '1',
  '3374.99',
  '269.9992'],
 ['SO43705',
  '1',
  '2019-07-01',
  'Curtis Lu',
  'curtis9@adventure-works.com',
  '"Mountain-100 Silver',
  ' 38"',
  '1',
  '3399.99',
  '271.9992'],
 ['SO43700',
  '1',
  '2019-07-01',
  'Ruben Prasad',
  'ruben10@adventure-works.com',
  '"Road-650 Black',
  ' 62"',
  '1',
  '699.0982',
  '55.9279'],
 ['SO43703',
  '1',
  '2019-07-01',
  'Albert Alvarez',
  'albert7@adventure-works.com',
  '"Road-150 Red',
  ' 62"',
  '1',
  '3578.27',
  '286.2616'],
 ['SO43697',
  '1',
  '2019-07-01',
  'Cole Watson',
  'cole1@adventure-

In [0]:
sales = sales.map(lambda r: r.split(','))
sales.take(2)

[['SalesOrderNumber',
  'SalesOrderLineNumber',
  'OrderDate',
  'CustomerName',
  'EmailAddress',
  'Item',
  'Quantity',
  'UnitPrice',
  'TaxAmount'],
 ['SO43701',
  '1',
  '2019-07-01',
  'Christy Zhu',
  'christy12@adventure-works.com',
  '"Mountain-100 Silver',
  ' 44"',
  '1',
  '3399.99',
  '271.9992']]

In [0]:
for row in sales.take(10):
    print(row[4])

EmailAddress
christy12@adventure-works.com
julio1@adventure-works.com
curtis9@adventure-works.com
ruben10@adventure-works.com
albert7@adventure-works.com
cole1@adventure-works.com
sydney61@adventure-works.com
colin45@adventure-works.com
rachael16@adventure-works.com


### flatMap
Similar to map, but each input item can be mapped to 0 or more output items (so func should return a Seq rather than a single item).

In [0]:
sc.parallelize([2, 3, 4]).flatMap(lambda x: [x,x,x]).collect()

[2, 2, 2, 3, 3, 3, 4, 4, 4]

In [0]:
sc.parallelize([2, 3, 4]).map(lambda x: [x,x,x]).collect()

[[2, 2, 2], [3, 3, 3], [4, 4, 4]]

### Filter

In [0]:
sales.count()

1202

In [0]:
sales.filter(lambda r: "Emma" in r[3]).count()

9

In [0]:
sales.filter(lambda r: "Emma" in r[3]).take(9)

[['SO43707',
  '1',
  '2019-07-02',
  'Emma Brown',
  'emma3@adventure-works.com',
  '"Road-150 Red',
  ' 48"',
  '1',
  '3578.27',
  '286.2616'],
 ['SO44344',
  '1',
  '2019-09-07',
  'Emmanuel Patel',
  'emmanuel3@adventure-works.com',
  '"Road-150 Red',
  ' 62"',
  '1',
  '3578.27',
  '286.2616'],
 ['SO44400',
  '1',
  '2019-09-14',
  'Emma Griffin',
  'emma66@adventure-works.com',
  '"Road-150 Red',
  ' 48"',
  '1',
  '3578.27',
  '286.2616'],
 ['SO44403',
  '1',
  '2019-09-15',
  'Emma Murphy',
  'emma32@adventure-works.com',
  '"Mountain-100 Black',
  ' 44"',
  '1',
  '3374.99',
  '269.9992'],
 ['SO44652',
  '1',
  '2019-10-14',
  'Emma Miller',
  'emma5@adventure-works.com',
  '"Mountain-100 Black',
  ' 44"',
  '1',
  '3374.99',
  '269.9992'],
 ['SO44717',
  '1',
  '2019-10-26',
  'Emma Sandberg',
  'emma47@adventure-works.com',
  '"Road-650 Black',
  ' 62"',
  '1',
  '699.0982',
  '55.9279'],
 ['SO45005',
  '1',
  '2019-11-26',
  'Emma Rivera',
  'emma34@adventure-works.com',
 

### mapPartitions
Similar to map, but runs separately on each partition (block) of the RDD, so func must be of type Iterator&lt;T&gt; =&gt; Iterator&lt;U&gt; when running on an RDD of type T

In [0]:
def f(i): yield sum(i)
parallel = sc.parallelize(range(1,10), 3)
parallel.mapPartitions(f).collect()

[6, 15, 24]

In [0]:
parallel = sc.parallelize(range(1,10))
parallel.mapPartitions(f).collect()

[3, 7, 11, 24]

### mapPartitionsWithIndex(func): 
Similar to mapPartitions, but also provides func with an integer value representing the index of the partition, so func must be of type (Int, Iterator&lt;T&gt;) =&gt; Iterator&lt;U&gt; when running on an RDD of type T

In [0]:
sales.take(3)

[['SalesOrderNumber',
  'SalesOrderLineNumber',
  'OrderDate',
  'CustomerName',
  'EmailAddress',
  'Item',
  'Quantity',
  'UnitPrice',
  'TaxAmount'],
 ['SO43701',
  '1',
  '2019-07-01',
  'Christy Zhu',
  'christy12@adventure-works.com',
  '"Mountain-100 Silver',
  ' 44"',
  '1',
  '3399.99',
  '271.9992'],
 ['SO43704',
  '1',
  '2019-07-01',
  'Julio Ruiz',
  'julio1@adventure-works.com',
  '"Mountain-100 Black',
  ' 48"',
  '1',
  '3374.99',
  '269.9992']]

In [0]:
def remove_header(idx, itr):
    return iter(list(itr)[1:]) if idx == 0 else itr

In [0]:
salesNoHeader = sales.mapPartitionsWithIndex(remove_header)
salesNoHeader.take(3)

[['SO43701',
  '1',
  '2019-07-01',
  'Christy Zhu',
  'christy12@adventure-works.com',
  '"Mountain-100 Silver',
  ' 44"',
  '1',
  '3399.99',
  '271.9992'],
 ['SO43704',
  '1',
  '2019-07-01',
  'Julio Ruiz',
  'julio1@adventure-works.com',
  '"Mountain-100 Black',
  ' 48"',
  '1',
  '3374.99',
  '269.9992'],
 ['SO43705',
  '1',
  '2019-07-01',
  'Curtis Lu',
  'curtis9@adventure-works.com',
  '"Mountain-100 Silver',
  ' 38"',
  '1',
  '3399.99',
  '271.9992']]

### Union
Return a new dataset that contains the union of the elements in the source dataset and the argument.

In [0]:
rdd_one = sc.parallelize(range(1,11))
rdd_two = sc.parallelize(range(5,16))

rdd_one.union(rdd_two).collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [0]:
sales19 = sc.textFile(source_data + '2019.csv')
sales20 = sc.textFile(source_data + '2020.csv')

In [0]:
print(sales19.count())
sales19.take(5)

1202


['SalesOrderNumber,SalesOrderLineNumber,OrderDate,CustomerName,EmailAddress,Item,Quantity,UnitPrice,TaxAmount',
 'SO43701,1,2019-07-01,Christy Zhu,christy12@adventure-works.com,"Mountain-100 Silver, 44",1,3399.99,271.9992',
 'SO43704,1,2019-07-01,Julio Ruiz,julio1@adventure-works.com,"Mountain-100 Black, 48",1,3374.99,269.9992',
 'SO43705,1,2019-07-01,Curtis Lu,curtis9@adventure-works.com,"Mountain-100 Silver, 38",1,3399.99,271.9992',
 'SO43700,1,2019-07-01,Ruben Prasad,ruben10@adventure-works.com,"Road-650 Black, 62",1,699.0982,55.9279']

In [0]:
print(sales20.count())
sales20.take(5)

2734


['SalesOrderNumber,SalesOrderLineNumber,OrderDate,CustomerName,EmailAddress,Item,Quantity,UnitPrice,TaxAmount',
 'SO45347,1,2020-01-01,Clarence Raji,clarence35@adventure-works.com,"Road-650 Black, 52",1,699.0982,55.9279',
 'SO45345,1,2020-01-01,Bonnie Yuan,bonnie12@adventure-works.com,"Road-150 Red, 52",1,3578.27,286.2616',
 'SO45348,1,2020-01-01,Leah Guo,leah14@adventure-works.com,"Road-150 Red, 44",1,3578.27,286.2616',
 'SO45349,1,2020-01-01,Candice Sun,candice19@adventure-works.com,"Road-150 Red, 48",1,3578.27,286.2616']

In [0]:
sales19.union(sales20).count()

3936

### Intersection
Return a new RDD that contains the intersection of elements in the source dataset and the argument.

In [0]:
rdd_one = sc.parallelize(range(1,11))
rdd_two = sc.parallelize(range(5,16))

sorted(rdd_one.intersection(rdd_two).collect())

[5, 6, 7, 8, 9, 10]

### Distinct
Return a new dataset that contains the distinct elements of the source dataset.

In [0]:
rdd_one = sc.parallelize(range(1,11))
rdd_two = sc.parallelize(range(5,16))

sorted(rdd_one.union(rdd_two).distinct().collect())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

### GroupByKey
When called on a dataset of (K, V) pairs, returns a dataset of (K, Iterable&lt;V&gt;) pairs

In [0]:
allsales = sc.textFile(source_data+'20*.csv')
header = allsales.first()

In [0]:
header

'SalesOrderNumber,SalesOrderLineNumber,OrderDate,CustomerName,EmailAddress,Item,Quantity,UnitPrice,TaxAmount'

In [0]:
allsales = allsales.filter(lambda r: r != header).map(lambda r: r.split(','))

In [0]:
allsales.take(2)

[['SO43701',
  '1',
  '2019-07-01',
  'Christy Zhu',
  'christy12@adventure-works.com',
  '"Mountain-100 Silver',
  ' 44"',
  '1',
  '3399.99',
  '271.9992'],
 ['SO43704',
  '1',
  '2019-07-01',
  'Julio Ruiz',
  'julio1@adventure-works.com',
  '"Mountain-100 Black',
  ' 48"',
  '1',
  '3374.99',
  '269.9992']]

In [0]:
rdd = allsales.map(lambda r: (r[2][:4], r)).groupByKey()

In [0]:
rdd.collect()

[('2019', <pyspark.resultiterable.ResultIterable at 0x7f75e0f22190>),
 ('2020', <pyspark.resultiterable.ResultIterable at 0x7f75e88e56d0>),
 ('2021', <pyspark.resultiterable.ResultIterable at 0x7f75e8d5b250>)]

In [0]:
rdd_19 = rdd.map(lambda r: {r[0]: list(r[1])}).take(1)
rdd_19

[{'2019': [['SO43701',
    '1',
    '2019-07-01',
    'Christy Zhu',
    'christy12@adventure-works.com',
    '"Mountain-100 Silver',
    ' 44"',
    '1',
    '3399.99',
    '271.9992'],
   ['SO43704',
    '1',
    '2019-07-01',
    'Julio Ruiz',
    'julio1@adventure-works.com',
    '"Mountain-100 Black',
    ' 48"',
    '1',
    '3374.99',
    '269.9992'],
   ['SO43705',
    '1',
    '2019-07-01',
    'Curtis Lu',
    'curtis9@adventure-works.com',
    '"Mountain-100 Silver',
    ' 38"',
    '1',
    '3399.99',
    '271.9992'],
   ['SO43700',
    '1',
    '2019-07-01',
    'Ruben Prasad',
    'ruben10@adventure-works.com',
    '"Road-650 Black',
    ' 62"',
    '1',
    '699.0982',
    '55.9279'],
   ['SO43703',
    '1',
    '2019-07-01',
    'Albert Alvarez',
    'albert7@adventure-works.com',
    '"Road-150 Red',
    ' 62"',
    '1',
    '3578.27',
    '286.2616'],
   ['SO43697',
    '1',
    '2019-07-01',
    'Cole Watson',
    'cole1@adventure-works.com',
    '"Road-150 Red',
   

In [0]:
len(rdd_19[0]['2019'])

1201

### reduceByKey
When called on a dataset of (K, V) pairs, returns a dataset of (K, V) pairs where the values for each key are aggregated using the given reduce function func, which must be of type (V,V) =&gt; V.

In [0]:
rdd = allsales.map(lambda r: (r[2][:4], 1)).reduceByKey(lambda v1, v2: v1+v2)

In [0]:
rdd.collect()

[('2019', 1201), ('2020', 2733), ('2021', 28784)]

### sortByKey([ascending], [numPartitions])
Sorts this RDD, which is assumed to consist of (key, value) pairs.

In [0]:
rdd.sortByKey().collect()

[('2019', 1201), ('2020', 2733), ('2021', 28784)]

In [0]:
rdd.sortByKey(False).collect()

[('2021', 28784), ('2020', 2733), ('2019', 1201)]

### Join

In [0]:
names1 = sc.parallelize(("car", "bus", "bike")).map(lambda a: (a, 1))
names2 = sc.parallelize(("bike", "cycle", "van")).map(lambda a: (a, 1))

In [0]:
names1.collect()

[('car', 1), ('bus', 1), ('bike', 1)]

In [0]:
names2.collect()

[('bike', 1), ('cycle', 1), ('van', 1)]

In [0]:
names1.join(names2).collect()

[('bike', (1, 1))]

In [0]:
names1.leftOuterJoin(names2).collect()

[('car', (1, None)), ('bus', (1, None)), ('bike', (1, 1))]

In [0]:
names1.rightOuterJoin(names2).collect()

[('van', (None, 1)), ('bike', (1, 1)), ('cycle', (None, 1))]